In [1]:
import re
import ast
import nltk
import numpy as np
import pandas as pd
from tqdm import tqdm
import pandas as pd
from nltk.corpus import stopwords
from natasha import Segmenter, MorphVocab, NewsEmbedding, NewsMorphTagger,Doc
from cleantext import clean
import nmslib

from sentence_transformers import SentenceTransformer


tqdm.pandas()
nltk.download("stopwords");

Since the GPL-licensed package `unidecode` is not installed, using Python's `unicodedata` package which yields worse results.
/mnt/88fdd009-dda3-49d8-9888-cfd9d9d5910a/ITMO/Vacancy-Recommendation-Service/.env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to /home/alex-
[nltk_data]     utk/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Очистка резюме

In [2]:
data = pd.read_csv('../DATA/dst-3.0_16_1_hh_database.csv', delimiter=';')
data['Опыт работы'] = data['Опыт работы'].astype(str)
data.head()

,"Пол, возраст",ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто
0,"Мужчина , 39 лет , родился 27 ноября 1979",29000 руб.,Системный администратор,"Советск (Калининградская область) , не готов к...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, ва...",Опыт работы 16 лет 10 месяцев Август 2010 — п...,"МАОУ ""СОШ № 1 г.Немана""",Системный администратор,Неоконченное высшее образование 2000 Балтийск...,16.04.2019 15:59,Имеется собственный автомобиль
1,"Мужчина , 60 лет , родился 20 марта 1959",40000 руб.,Технический писатель,"Королев , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, уд...",Опыт работы 19 лет 5 месяцев Январь 2000 — по...,Временный трудовой коллектив,"Менеджер проекта, Аналитик, Технический писатель",Высшее образование 1981 Военно-космическая ак...,12.04.2019 08:42,Не указано
2,"Женщина , 36 лет , родилась 12 августа 1982",20000 руб.,Оператор,"Тверь , не готова к переезду , не готова к ком...",полная занятость,полный день,Опыт работы 10 лет 3 месяца Октябрь 2004 — Де...,ПАО Сбербанк,Кассир-операционист,Среднее специальное образование 2002 Профессио...,16.04.2019 08:35,Не указано
3,"Мужчина , 38 лет , родился 25 июня 1980",100000 руб.,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,"Саратов , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, удаленная работа",Опыт работы 18 лет 9 месяцев Август 2017 — Ап...,OpenSoft,Инженер-программист,Высшее образование 2002 Саратовский государст...,08.04.2019 14:23,Не указано
4,"Женщина , 26 лет , родилась 3 марта 1993",140000 руб.,Региональный менеджер по продажам,"Москва , не готова к переезду , готова к коман...",полная занятость,полный день,Опыт работы 5 лет 7 месяцев Региональный мене...,Мармелад,Менеджер по продажам,Высшее образование 2015 Кгу Психологии и педаг...,22.04.2019 10:32,Не указано


In [3]:
segmenter = Segmenter()
morph_vocab = MorphVocab()
morph_tagger = NewsMorphTagger(NewsEmbedding())

ru_stopwords = stopwords.words("russian")
ru_stopwords.extend(['-', 'год', 'настоящий', 'время', 'месяц', 
                     'январь', 'февраль', 'март', 'апрель', 'май',
                     'июнь', 'июль', 'август', 'сентябрь', 'октябрь',
                     'ноябрь', 'декабрь', 'опыт', 'работа'])
allowed_symbols = re.compile(u'[0-9]|[^\u0400-\u04FFa-zA-Z \-]')

def normalise(text):
    text = clean(text,
        lower=True,
        fix_unicode=False,
        to_ascii=False,
        no_line_breaks=False,           
        no_urls=True,                 
        no_emails=True,              
        no_phone_numbers=True,        
        no_numbers=True,             
        no_digits=True,              
        no_punct=False,            
        replace_with_url=" ",
        replace_with_email=" ",
        replace_with_phone_number=" ",
        replace_with_number=" ",
        replace_with_digit=" ",
        replace_with_currency_symbol=" "
    )
    text = allowed_symbols.sub(' ', text)
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    
    
    filtered_tokens = [token.lemma
                       for token
                       in doc.tokens]
                    #    if token.pos in ['NOUN', 'ADJ', 'VERB', 'ADV']]
    
    filtered_tokens = list(filter(lambda token: len(token) > 1 and token not in ru_stopwords, filtered_tokens))
    
    return filtered_tokens

data['Опыт работы'] = data['Опыт работы'].progress_apply(normalise)
data.to_pickle('../DATA/hh_resume_processed.pkl') 

100%|██████████| 44744/44744 [37:15<00:00, 20.02it/s] 


### Эмбеддинги резюме

In [2]:
data_resume = pd.read_pickle('../DATA/hh_resume_processed.pkl')
data_resume['Ищет работу на должность:'] = data_resume['Ищет работу на должность:'].astype(str)
data_resume['Последняя/нынешняя должность'] = data_resume['Последняя/нынешняя должность'].astype(str)
# data['Опыт работы'] = data['Опыт работы'].astype(str)
data_resume.head()

,"Пол, возраст",ЗП,Ищет работу на должность:,"Город, переезд, командировки",Занятость,График,Опыт работы,Последнее/нынешнее место работы,Последняя/нынешняя должность,Образование и ВУЗ,Обновление резюме,Авто
0,"Мужчина , 39 лет , родился 27 ноября 1979",29000 руб.,Системный администратор,"Советск (Калининградская область) , не готов к...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, ва...","[маоу, сош, неман, системный, администратор, о...","МАОУ ""СОШ № 1 г.Немана""",Системный администратор,Неоконченное высшее образование 2000 Балтийск...,16.04.2019 15:59,Имеется собственный автомобиль
1,"Мужчина , 60 лет , родился 20 марта 1959",40000 руб.,Технический писатель,"Королев , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, полный день, сменный график, уд...","[временный, трудовой, коллектив, москва, инфор...",Временный трудовой коллектив,"Менеджер проекта, Аналитик, Технический писатель",Высшее образование 1981 Военно-космическая ак...,12.04.2019 08:42,Не указано
2,"Женщина , 36 лет , родилась 12 августа 1982",20000 руб.,Оператор,"Тверь , не готова к переезду , не готова к ком...",полная занятость,полный день,"[пао, сбербанк, кассир-операционистый, денежны...",ПАО Сбербанк,Кассир-операционист,Среднее специальное образование 2002 Профессио...,16.04.2019 08:35,Не указано
3,"Мужчина , 38 лет , родился 25 июня 1980",100000 руб.,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,"Саратов , не готов к переезду , готов к редким...","частичная занятость, проектная работа, полная ...","гибкий график, удаленная работа","[opensoft, саратов, информационный, технология...",OpenSoft,Инженер-программист,Высшее образование 2002 Саратовский государст...,08.04.2019 14:23,Не указано
4,"Женщина , 26 лет , родилась 3 марта 1993",140000 руб.,Региональный менеджер по продажам,"Москва , не готова к переезду , готова к коман...",полная занятость,полный день,"[региональный, менеджер, продажа, руб, информа...",Мармелад,Менеджер по продажам,Высшее образование 2015 Кгу Психологии и педаг...,22.04.2019 10:32,Не указано


In [3]:
encoder = SentenceTransformer('cointegrated/rubert-tiny2')
encoder.cuda()

SentenceTransformer(
  (0): Transformer({'max_seq_length': 2048, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 312, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
)

In [4]:
type(encoder)

sentence_transformers.SentenceTransformer.SentenceTransformer

In [8]:
encoder = SentenceTransformer('cointegrated/rubert-tiny2')
encoder.cuda()

def preprocess_resumes(row):  
    
    all_info =  [row['Ищет работу на должность:'], row['Последняя/нынешняя должность'], *row['Опыт работы']]
    all_info_str = ' '.join(all_info)
    return encoder.encode(all_info_str, normalize_embeddings=True)

data_nesessary = data_resume.progress_apply(preprocess_resumes, axis=1)

resume_embeddings = np.stack(data_nesessary)
np.savez_compressed('resume_embeddings.npz', resume_embeddings=resume_embeddings)

100%|██████████| 44744/44744 [10:18<00:00, 72.30it/s] 


### Эмбеддинги вакансий

In [3]:
data_vacancy = pd.read_pickle('/mnt/88fdd009-dda3-49d8-9888-cfd9d9d5910a/ITMO/rabota_ru_parser/hh_data_processed.pkl')
data_vacancy.head()

,id,custom_position,description,schedule,experience,employment,salary_from,salary_to,salary_pay_type,city_name,city_id,published_date,work_skills,professional_roles,professional_roles_id,region_id,industry_name,industry_id
0,87757256,"Товаровед Магнит Косметик (Карла Маркса, 88)","[широкий, сеть, магнит, косметика, это, возмож...",Сменный график,От 1 года до 3 лет,Полная занятость,34000.0,39500.0,True,Белев,1915,2023-11-01T07:54:12+0300,[],Товаровед,127,1913,['Розничная торговля'],['2']
1,88926254,Ведущий специалист Группы товародвижения,"[обязанность, обеспечение, товародвижение, тер...",Полный день,От 1 года до 3 лет,Полная занятость,NaN,37000.0,True,Тула,92,2023-11-01T11:01:44+0300,"['Товароведение', 'Учет остатков', '1С: Предпр...",Товаровед,127,1913,['Розничная торговля'],['2']
2,89396431,"Товаровед (Красноармейская, 1)","[широкий, сеть, магазин, магнит, это, просто, ...",Сменный график,От 1 года до 3 лет,Полная занятость,38300.0,42600.0,True,Советск (Тульская область),1926,2023-11-13T17:24:03+0300,[],Товаровед,127,1913,['Розничная торговля'],['2']
3,87763816,"Товаровед Магнит Косметик (Шоссейная, 1)","[широкий, сеть, магнит, косметика, это, возмож...",Сменный график,От 1 года до 3 лет,Полная занятость,42000.0,49000.0,True,Барсуки,4039,2023-11-22T11:39:33+0300,[],Товаровед,127,1913,['Розничная торговля'],['2']
4,86397590,"Ночной товаровед (Фрунзе, 7Б)","[широкий, сеть, магазин, магнит, это, просто, ...",Сменный график,От 1 года до 3 лет,Полная занятость,48600.0,59700.0,True,Южный (Тульская область),10280,2023-11-22T11:24:37+0300,[],Товаровед,127,1913,['Розничная торговля'],['2']


In [3]:
encoder = SentenceTransformer('cointegrated/rubert-tiny2')
encoder.cuda()

def preprocess(row):   
    all_info =  [*row['industry_name'], row['professional_roles'], *row['work_skills'], *row['description']]
    all_info_str = ' '.join(all_info)
    return encoder.encode(all_info_str, normalize_embeddings=True)

data_nesessary: pd.DataFrame = data_vacancy.loc[:, ['description', 'work_skills', 'professional_roles', 'industry_name']]
data_nesessary['embedding'] = data_nesessary.progress_apply(preprocess, axis=1)

vacancy_embeddings = np.stack(data_nesessary['embedding'])
np.savez_compressed('vacancy_embeddings.npz', vacancy_embeddings=vacancy_embeddings)

100%|██████████| 140197/140197 [11:55<00:00, 195.92it/s]


### Ресерч

In [2]:
data_resume = pd.read_pickle('../DATA/hh_resume_processed.pkl')
data_vacancy = pd.read_pickle('/mnt/88fdd009-dda3-49d8-9888-cfd9d9d5910a/ITMO/rabota_ru_parser/hh_data_processed.pkl')

In [4]:
embeddings = np.load('../DATA/all_embeddings_rubert.npz')
resume_embeddings = embeddings['resume_embeddings']
vacancy_embeddings = embeddings['vacancy_embeddings']

del embeddings
resume_embeddings.shape, vacancy_embeddings.shape

((44744, 312), (140197, 312))

In [5]:
params = {
    'indexThreadQty':8,
    'M': 45,
    'efConstruction': 70
}
vacnacies = nmslib.init(method='hnsw', space='cosinesimil', data_type=nmslib.DataType.DENSE_VECTOR)
vacnacies.addDataPointBatch(vacancy_embeddings)
vacnacies.createIndex(params, print_progress=True)


0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************


In [6]:
vacnacies.saveIndex('/mnt/88fdd009-dda3-49d8-9888-cfd9d9d5910a/ITMO/Vacancy-Recommendation-Service/DATA/vacnacies/vacancy_index', save_data=True)

In [7]:
vacnacies.loadIndex('/mnt/88fdd009-dda3-49d8-9888-cfd9d9d5910a/ITMO/Vacancy-Recommendation-Service/DATA/vacnacies/vacancy_index', load_data=True)

In [8]:
encoder = SentenceTransformer('cointegrated/rubert-tiny2')

embedd = encoder.encode('Фитнесс тренер в спортзал', normalize_embeddings=True)
vacancy_ids, distances = vacnacies.knnQuery(embedd, k=7)
display(data_vacancy.loc[vacancy_ids])

,id,custom_position,description,schedule,experience,employment,salary_from,salary_to,salary_pay_type,city_name,city_id,published_date,work_skills,professional_roles,professional_roles_id,region_id,industry_name,industry_id
28158,89028044,Тренер по фитнесу,"[приглашать, постоянный, место, работа, инстру...",Полный день,От 1 года до 3 лет,Полная занятость,50000.0,50000.0,True,Вологда,25,2023-11-03T08:49:57+0300,[],"Фитнес-тренер, инструктор тренажерного зала",138,1739,"[Спортивные клубы, фитнес, салоны красоты]",['21']
82788,89231726,Тренер в тренажерный зал,"[обязанность, дежурный, час, проведение, персо...",Полный день,От 1 года до 3 лет,Полная занятость,25000.0,150000.0,True,Красноярск,54,2023-11-09T08:05:38+0300,[],"Фитнес-тренер, инструктор тренажерного зала",138,1146,"[Спортивные клубы, фитнес, салоны красоты]",['21']
5797,88972517,Инструктор по спорту (вид спорта-картинг),"[обязанность, организовывать, проводить, оздор...",Полный день,Нет опыта,Полная занятость,35734.0,NaN,True,Когалым,1370,2023-11-02T08:01:06+0300,[],"Фитнес-тренер, инструктор тренажерного зала",138,1368,"[Спортивные клубы, фитнес, салоны красоты]",['21']
8235,88612041,Инструктор по йоге,"[проведение, тренировочный, занятие, йога, спо...",Сменный график,От 1 года до 3 лет,Частичная занятость,18500.0,NaN,False,Челябинск,104,2023-10-25T07:57:41+0300,[],"Фитнес-тренер, инструктор тренажерного зала",138,1384,"[Спортивные клубы, фитнес, салоны красоты]",['21']
113627,89644923,Тренер по футболу,"[обязанность, проводить, тренировка, футбол, р...",Полный день,Нет опыта,Частичная занятость,15000.0,25000.0,False,Хабаровск,102,2023-11-20T09:15:15+0300,[],"Фитнес-тренер, инструктор тренажерного зала",138,1975,"[Спортивные клубы, фитнес, салоны красоты]",['21']
14834,88578705,Тренер по футболу,"[обязанность, организация, работа, платный, кр...",Полный день,Нет опыта,Частичная занятость,3000.0,7000.0,False,Тверь,89,2023-10-24T12:20:28+0300,[],"Фитнес-тренер, инструктор тренажерного зала",138,1783,"[Спортивные клубы, фитнес, салоны красоты]",['21']
55225,88759620,Инструктор тренажёрного зала,"[обязанность, контроль, посетитель, тренажерны...",Полный день,Нет опыта,Полная занятость,25000.0,NaN,True,Липецк,58,2023-10-27T15:14:23+0300,[],"Фитнес-тренер, инструктор тренажерного зала",138,1890,"[Спортивные клубы, фитнес, салоны красоты]",['21']


In [23]:
resume_id = 10
vacancy_ids, distances = vacnacies.knnQuery(resume_embeddings[resume_id], k=7)

display(data_resume.loc[resume_id])
display(data_vacancy.loc[vacancy_ids])

Пол, возраст                             Женщина ,  35 лет , родилась 29 ноября 1983
ЗП                                                                        55000 руб.
Ищет работу на должность:                      Менеджер / оператор интернет-магазина
Город, переезд, командировки       Москва , не готова к переезду , готова к коман...
Занятость                                                           полная занятость
График                                                                   полный день
Опыт работы                        [сеть, свадебный, салон, primavera, primavera,...
Последнее/нынешнее место работы                       Сеть свадебных салон Primavera
Последняя/нынешняя должность                Администратор интернет-магазина и салона
Образование и ВУЗ                  Среднее специальное образование 2002 Торгово-т...
Обновление резюме                                                   11.04.2019 10:56
Авто                                                             

,id,custom_position,description,schedule,experience,employment,salary_from,salary_to,salary_pay_type,city_name,city_id,published_date,work_skills,professional_roles,professional_roles_id,region_id,industry_name,industry_id
11503,89142030,Консультант интернет-магазина,"[привет, звать, лика, hr-менеджер, компания, i...",Удаленная работа,Нет опыта,Полная занятость,35000.0,47000.0,False,Иваново (Ивановская область),32,2023-11-07T13:25:22+0300,"['Пользователь ПК', 'Грамотная речь']",Диспетчер,39,1754,"['Транспорт, логистика, перевозки']",['10']
94009,89142025,Консультант интернет-магазина,"[привет, звать, лика, hr-менеджер, компания, i...",Удаленная работа,Нет опыта,Полная занятость,35000.0,47000.0,False,Киров (Кировская область),49,2023-11-07T13:25:22+0300,"['Пользователь ПК', 'Грамотная речь']",Диспетчер,39,1661,"['Транспорт, логистика, перевозки']",['10']
72176,89142020,Консультант интернет-магазина,"[привет, звать, лика, hr-менеджер, компания, i...",Удаленная работа,Нет опыта,Полная занятость,35000.0,47000.0,False,Томск,90,2023-11-07T13:25:22+0300,"['Пользователь ПК', 'Грамотная речь']",Диспетчер,39,1255,"['Транспорт, логистика, перевозки']",['10']
14911,89142033,Консультант интернет-магазина,"[привет, звать, лика, hr-менеджер, компания, i...",Удаленная работа,Нет опыта,Полная занятость,35000.0,47000.0,False,Тверь,89,2023-11-07T13:25:22+0300,"['Пользователь ПК', 'Грамотная речь']",Диспетчер,39,1783,"['Транспорт, логистика, перевозки']",['10']
84457,89142032,Консультант интернет-магазина,"[привет, звать, лика, hr-менеджер, компания, i...",Удаленная работа,Нет опыта,Полная занятость,35000.0,47000.0,False,Улан-Удэ,20,2023-11-07T13:25:22+0300,"['Пользователь ПК', 'Грамотная речь']",Диспетчер,39,1118,"['Транспорт, логистика, перевозки']",['10']
17044,89142010,Консультант интернет-магазина,"[привет, звать, лика, hr-менеджер, компания, i...",Удаленная работа,Нет опыта,Полная занятость,35000.0,47000.0,False,Ярославль,112,2023-11-07T13:25:22+0300,"['Пользователь ПК', 'Грамотная речь']",Диспетчер,39,1806,"['Транспорт, логистика, перевозки']",['10']
90468,89142013,Консультант интернет-магазина,"[привет, звать, лика, hr-менеджер, компания, i...",Удаленная работа,Нет опыта,Полная занятость,35000.0,47000.0,False,Махачкала,29,2023-11-07T13:25:22+0300,"['Пользователь ПК', 'Грамотная речь']",Диспетчер,39,1424,"['Транспорт, логистика, перевозки']",['10']


In [9]:
params = {
    'indexThreadQty':8,
    'M': 45,
    'efConstruction': 70
}
resumes = nmslib.init(method='hnsw', space='cosinesimil', data_type=nmslib.DataType.DENSE_VECTOR)
resumes.addDataPointBatch(resume_embeddings)
resumes.createIndex(params, print_progress=True)


0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************


In [ ]:
resumes.saveIndex('/mnt/88fdd009-dda3-49d8-9888-cfd9d9d5910a/ITMO/Vacancy-Recommendation-Service/DATA/resumes/resume_index', save_data=True)

### Гипотеза bag_of_words

In [2]:
data_resume = pd.read_pickle('../DATA/hh_resume_processed.pkl')
data_resume['Ищет работу на должность:'] = data_resume['Ищет работу на должность:'].astype(str)
data_resume['Последняя/нынешняя должность'] = data_resume['Последняя/нынешняя должность'].astype(str)

def preprocess_resumes(row):  
    all_info =  [row['Ищет работу на должность:'], row['Последняя/нынешняя должность'], *row['Опыт работы']]
    all_info_str = ' '.join(all_info)
    return all_info_str

all_possible_texts = data_resume.progress_apply(preprocess_resumes, axis=1)
del data_resume

100%|██████████| 44744/44744 [00:01<00:00, 40854.73it/s]


In [4]:
data_vacancy = pd.read_pickle('/mnt/88fdd009-dda3-49d8-9888-cfd9d9d5910a/ITMO/rabota_ru_parser/hh_data_processed.pkl')

def preprocess_vacancy(row):   
    all_info =  [*row['industry_name'], row['professional_roles'], *row['work_skills'], *row['description']]
    all_info_str = ' '.join(all_info)
    return all_info_str

vac_description = data_vacancy.progress_apply(preprocess_vacancy, axis=1)
del data_vacancy

all_possible_texts = pd.concat((all_possible_texts, vac_description))

100%|██████████| 140197/140197 [00:02<00:00, 59702.73it/s]


In [6]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(all_possible_texts)
# vectorizer.get_feature_names_out()
# X.toarray()

CountVectorizer()

In [8]:
data_vacancy = pd.read_pickle('/mnt/88fdd009-dda3-49d8-9888-cfd9d9d5910a/ITMO/rabota_ru_parser/hh_data_processed.pkl')

def preprocess_vacancy(row):   
    all_info =  [*row['industry_name'], row['professional_roles'], *row['work_skills'], *row['description']]
    all_info_str = ' '.join(all_info)
    return all_info_str

vac_description = data_vacancy.progress_apply(preprocess_vacancy, axis=1)
del data_vacancy
vac_description_vectores = vectorizer.transform(vac_description)

100%|██████████| 140197/140197 [00:02<00:00, 50286.19it/s]


In [9]:
data_resume = pd.read_pickle('../DATA/hh_resume_processed.pkl')
data_resume['Ищет работу на должность:'] = data_resume['Ищет работу на должность:'].astype(str)
data_resume['Последняя/нынешняя должность'] = data_resume['Последняя/нынешняя должность'].astype(str)

def preprocess_resumes(row):  
    all_info =  [row['Ищет работу на должность:'], row['Последняя/нынешняя должность'], *row['Опыт работы']]
    all_info_str = ' '.join(all_info)
    return all_info_str

resume_description = data_resume.progress_apply(preprocess_resumes, axis=1)
del data_resume
resume_description_vectores = vectorizer.transform(resume_description)

100%|██████████| 44744/44744 [00:01<00:00, 37667.24it/s]


In [38]:
params = {
    'indexThreadQty':8,
    # 'M': 45,
    'efConstruction': 70
}

vacnacies = nmslib.init(method='sw-graph', space='l2_sparse', data_type=nmslib.DataType.SPARSE_VECTOR)
vacnacies.addDataPointBatch(vac_description_vectores)
vacnacies.createIndex(params, print_progress=True)


0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************


In [39]:
resume_description_vectores.shape, vac_description_vectores.shape

((44744, 252875), (140197, 252875))

In [40]:
a = resume_description_vectores[resume_id]
a

<1x252875 sparse matrix of type '<class 'numpy.int64'>'
	with 28 stored elements in Compressed Sparse Row format>

In [42]:
resume_id = 0
vacnacies.knnQuery(a, k=1)

# display(data_resume.loc[resume_id])
# display(data_vacancy.loc[vacancy_ids])

IndexError: tuple index out of range

----

In [4]:
import pandas as pd
from sqlalchemy import create_engine, Table, Column, Integer, \
                       String, MetaData
from sqlalchemy.orm import Session
from sqlalchemy.orm import DeclarativeBase
   
# строка подключения
sqlite_database = "sqlite:///test.db"
   
# создаем движок SqlAlchemy
engine = create_engine(sqlite_database)

#создаем базовый класс для моделей
class Base(DeclarativeBase): pass
  
# создаем модель, объекты которой будут храниться в бд
class Vacancies(Base):
    __tablename__ = "vacancies"
    
    index = Column(Integer, primary_key=True, index=True)
    custom_position = Column(String)
    experience = Column(Integer)
    salary = Column(Integer)
    city_name = Column(String)
  
# создаем таблицы
Base.metadata.create_all(bind=engine)

data_vacancy.to_sql('vacancies', if_exists='replace', con=engine, index=True)

customer_data_from_db = pd.read_sql_table('vacancies', engine)
customer_data_from_db

,index,custom_position,experience,salary,city_name
0,0,"Товаровед Магнит Косметик (Карла Маркса, 88)",2,34000,Белев
1,1,Ведущий специалист Группы товародвижения,2,50000,Тула
2,2,"Товаровед (Красноармейская, 1)",2,38300,Советск (Тульская область)
3,3,"Товаровед Магнит Косметик (Шоссейная, 1)",2,42000,Барсуки
4,4,"Ночной товаровед (Фрунзе, 7Б)",2,48600,Южный (Тульская область)
...,...,...,...,...,...
140192,140192,Энергетик,4,60000,Тюмень
140193,140193,Ведущий инженер-энергетик,4,75000,Тюмень
140194,140194,"Инженер-энергетик (ХМАО, Урай)",4,150000,Тюмень
140195,140195,Энергетик,4,88000,Тюмень


In [1]:
import pandas as pd
from sqlalchemy import create_engine, Table, Column, Integer, \
                       String, MetaData
from sqlalchemy.orm import Session
from sqlalchemy.orm import DeclarativeBase

sqlite_database = "sqlite:////mnt/88fdd009-dda3-49d8-9888-cfd9d9d5910a/ITMO/Vacancy-Recommendation-Service/test.db" 
   
# создаем движок SqlAlchemy
engine = create_engine(sqlite_database)

In [5]:
class Base(DeclarativeBase): pass

class Vacancies(Base):
    __tablename__ = "vacancies"

    index = Column(Integer, primary_key=True, index=True)
    custom_position = Column(String)
    experience = Column(Integer)
    salary = Column(Integer)
    city_name = Column(String)

class Resumes(Base):
    __tablename__ = "resumes"

    index = Column(Integer, primary_key=True, index=True)
    description = Column(String)
    salary = Column(Integer)
    vacancy = Column(String)
    education = Column(String)
    city_name = Column(String)

In [6]:
idxes = [2, 10, 20, 33 ,80]

with Session(autoflush=False, bind=engine) as db:
    # получение всех объектов
    q = db.query(Vacancies).filter(Vacancies.index.in_(idxes))
    result = q.all()
    for res in q:
        print(f"{res.index}\t{res.custom_position}")
print('----')
with Session(autoflush=False, bind=engine) as db:
    # получение всех объектов
    q = db.query(Resumes).filter(Resumes.index.in_(idxes))
    result = q.all()
    for res in q:
        print(f"{res.index}\t{res.description}")

2	Товаровед (Красноармейская, 1)
10	Ночной товаровед (Болдина, 58А)
20	Агроном консультант
33	Массажист
80	Инженер ПТО
----
2	Женщина ,  36 лет , родилась 12 августа 1982
10	Женщина ,  35 лет , родилась 29 ноября 1983
20	Мужчина ,  31 год , родился 30 января 1988
33	Женщина ,  23 года , родилась 1 сентября 1995
80	Мужчина ,  25 лет , родился 4 марта 1994


In [9]:
metadata = MetaData()
vacancies_table = Table(
    'vacancies', metadata,
    Column('index', Integer, primary_key=True, index=True),
    Column('custom_position', String),
    Column('experience', String),
    Column('salary', Integer),
    Column('city_name', String)
)

In [15]:
ids = [0, 10, 25]
engine = create_engine("sqlite:///test.db")
res = vacancies_table.select().where('index' in ids).compile(engine)

In [17]:
res.string

'SELECT vacancies."index", vacancies.custom_position, vacancies.experience, vacancies.salary, vacancies.city_name \nFROM vacancies \nWHERE 0 = 1'

In [ ]:
import pandas as pd
from sqlalchemy import create_engine, Table, Column, Integer, \
                       String, MetaData
from sqlalchemy.orm import DeclarativeBase

# создаем движок SqlAlchemy
engine = create_engine("sqlite:///test.db")


data_vacancy.to_sql('vacancies_table', con=engine, if_exists='replace', index=True)

customer_data_from_db = pd.read_sql_table('vacancies_table', engine)

In [7]:
customer_data_from_db

,index,custom_position,experience,salary,city_name
0,0,"Товаровед Магнит Косметик (Карла Маркса, 88)",2,34000,Белев
1,1,Ведущий специалист Группы товародвижения,2,50000,Тула
2,2,"Товаровед (Красноармейская, 1)",2,38300,Советск (Тульская область)
3,3,"Товаровед Магнит Косметик (Шоссейная, 1)",2,42000,Барсуки
4,4,"Ночной товаровед (Фрунзе, 7Б)",2,48600,Южный (Тульская область)
...,...,...,...,...,...
140192,140192,Энергетик,4,60000,Тюмень
140193,140193,Ведущий инженер-энергетик,4,75000,Тюмень
140194,140194,"Инженер-энергетик (ХМАО, Урай)",4,150000,Тюмень
140195,140195,Энергетик,4,88000,Тюмень


In [2]:
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine("sqlite:////mnt/88fdd009-dda3-49d8-9888-cfd9d9d5910a/ITMO/Vacancy-Recommendation-Service/vacancies_and_resumes.db")

customer_data_from_db = pd.read_sql_table('resumes', engine)
customer_data_from_db.head()

,index,description,salary,vacancy,education,city_name
0,0,"Мужчина , 39 лет , родился 27 ноября 1979",29000,Системный администратор,Неоконченное высшее образование 2000 Балтийск...,Советск (Калининградская область)
1,1,"Мужчина , 60 лет , родился 20 марта 1959",40000,Технический писатель,Высшее образование 1981 Военно-космическая ак...,Королев
2,2,"Женщина , 36 лет , родилась 12 августа 1982",20000,Оператор,Среднее специальное образование 2002 Профессио...,Тверь
3,3,"Мужчина , 38 лет , родился 25 июня 1980",100000,Веб-разработчик (HTML / CSS / JS / PHP / базы ...,Высшее образование 2002 Саратовский государст...,Саратов
4,4,"Женщина , 26 лет , родилась 3 марта 1993",140000,Региональный менеджер по продажам,Высшее образование 2015 Кгу Психологии и педаг...,Москва
